In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [52]:
class Layer(object):
    """define a layer object"""
    
    def __init__(self, n_input, n_output=None, random_seed=42):
        """Constructer of a layer object"""
        
        self.n_input = n_input
        self.n_output = n_output
        
        np.random.seed(random_seed)
        self.weights = np.random.rand(self.n_input, self.n_output)

    def _sigmoid_forward(self, x):
        """Apply sigmoid function"""
        
        return 1 / (1 + np.exp(-x))
    
    def forward_prop(self, input_x, activation_func='sigmoid'):
        """Implement forward propagation"""
        
        if activation_func == 'sigmoid':
            self.out_x = self._sigmoid_forward(input_x.dot(self.weights))
    
    def backward_prop(self, out_x, delta_next, weights_next, eta):
        """Implement backward propagation"""
        
        self.delta = out_x.dot((1-out_x).T).dot(delta_next).dot(weights_next.T)
        
        self.weights -= eta * out_x.T.dot(self.delta)
        
class Output_Layer(Layer):
    """define a output layer object"""
    
    def __init__(self, n_input, n_output=None, random_seed=42):
        """Constructer of a output layer object"""
            
        Layer.__init__(self, n_input, n_output, random_seed)
    
    def _softmax(self, out_x):
        return np.exp(out_x) / np.sum(np.exp(out_x), axis=1)
    
    def backward_prop(self, y_true, y_preds, out_x, eta):
        """Implement backward propagation (output layer)"""
        
        self.delta = 2 * (y_true - y_preds)
        
        self.weights -= eta * out_x.T.dot(self.delta)
    
    def predict(self, y_true):
        """Predict labels"""
        
        # self.pred_prop = self._softmax(self.out_x)
        return np.where(self.out_x > 0.5, 1, 0)
    
    def compute_mse(self, y_true):
        return np.mean(np.square(self.out_x - y_true))

In [53]:
X = np.array([[1, 0, 0], [1, 1, 0], [1, 0, 1], [1, 1, 1]])
y = np.array([1, 0, 0, 1])
y = y.reshape(4, 1)

In [54]:
epochs = 10
n_neurons = 3
fc_layer = Layer(X.shape[1], n_neurons)
output_layer = Output_Layer(n_neurons, 1)

In [55]:
for epoch in range(epochs):
    fc_layer.forward_prop(X)
    output_layer.forward_prop(fc.out_x)
    
    y_preds = output_layer.predict(y)
    print(output_layer.compute_mse(y))

    output_layer.backward_prop(y, y_preds, fc_layer.out_x, 0.1)
    fc_layer.backward_prop(out_x=X, delta_next=output_layer.delta, weights_next=output_layer.weights, eta=0.1)

0.3575064513046198
0.4124304435143616
0.45630217084143715
0.48062813672561727
0.4918013196014277
0.49658563539638756
0.4985866902717198
0.4994164078579353
0.49975924473621613
0.49990070795055674
